In [ ]:
#V2 will check if the path is empty in the csv. 
#If empty and available fill it,not empty not rewrite it
#v3 do it withoyt RT STRuctBuilder
#v4 again with RTStruct Builder,  because v3 was incomplete, but now it does it with both ITV and GTVs labels separately
#v5, same as v4 but clean

In [ ]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os
import csv
import re
import pandas as pd

In [ ]:
def LookForLabelMatch(ListOfNamesinRT,LabelsToLookFor,currPathTemp):
    tempList = []
    for labeltot in LabelsToLookFor:
        if labeltot in ListOfNamesinRT:
            tempList.append(currPathTemp)
            return tempList
    return tempList

In [ ]:
def GetTheContourRTMatch(ctfolder,rtfolder,itv_tot_labels,itv_tumor_labels,itv_ln_labels,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels):
    singlematch = False # If one RT file has already been matched you dont need to loop to the next one. In theory only one RTStruct should be necessary
    ct_Match = []
    totGTV_Match, tumorGTV_Match, lnGTV_Match = ([],[],[])
    totITV_Match, tumorITV_Match, lnITV_Match = ([],[],[])
    gtv_bool = False
    itv_bool = False
    if len(ctfolder)==0:
        print("No CT")
    for k in range(len(ctfolder)):
        for j in range(len(rtfolder)):
            rtfiles = os.listdir(rtfolder[j])
            for i in range(len(rtfiles)):
                if not singlematch:
                    try:
                        print("check",i,j,k)
                        
                        rtstruct = RTStructBuilder.create_from(dicom_series_path=ctfolder[k],rt_struct_path=str(rtfolder[j]+rtfiles[i]))

                        totITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_tot_labels,str(rtfolder[j]+rtfiles[i]))
                        tumorITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_tumor_labels,str(rtfolder[j]+rtfiles[i]))
                        lnITV_Match = LookForLabelMatch(rtstruct.get_roi_names(),itv_ln_labels,str(rtfolder[j]+rtfiles[i]))

                        totGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_tot_labels,str(rtfolder[j]+rtfiles[i]))
                        tumorGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_tumor_labels,str(rtfolder[j]+rtfiles[i]))
                        lnGTV_Match = LookForLabelMatch(rtstruct.get_roi_names(),gtv_ln_labels,str(rtfolder[j]+rtfiles[i]))

                        if len(totITV_Match) ==0 and len(tumorITV_Match)+len(lnITV_Match) == 0:
                            print("No labels Matched for ITV",rtfiles[i],rtstruct.get_roi_names())
                        else: itv_bool = True
                        
                        if len(totGTV_Match) ==0 and len(tumorGTV_Match)+len(lnGTV_Match) == 0:
                            print("No labels Matched for GTV",rtfiles[i],rtstruct.get_roi_names())
                        else: gtv_bool = True
                        
                        if (len(totITV_Match)>0 or len(tumorITV_Match)+len(lnITV_Match)>0) or (len(totGTV_Match)>0 or len(tumorGTV_Match)+len(lnGTV_Match) >0):
                            ct_Match.append(ctfolder[k])
                            print("M ITV","tot",len(totITV_Match),"tumor",len(tumorITV_Match),"LN",len(lnITV_Match))
                            print("M GTV","tot",len(totGTV_Match),"tumor",len(tumorGTV_Match),"LN",len(lnGTV_Match))
                            if len(totITV_Match) == 0: totITV_Match.append('0')
                            if len(tumorITV_Match) == 0: tumorITV_Match.append('0')
                            if len(lnITV_Match) == 0: lnITV_Match.append('0')
                            if len(totGTV_Match) == 0: totGTV_Match.append('0')
                            if len(tumorGTV_Match) == 0: tumorGTV_Match.append('0')
                            if len(lnGTV_Match) == 0: lnGTV_Match.append('0')
                                
                        if gtv_bool and itv_bool:
                            singlematch = True
                    except Exception as e: print(e)#pass

        #mask_3d = rtstruct.get_roi_mask_by_name(label)
    return ct_Match,totITV_Match, tumorITV_Match, lnITV_Match,totGTV_Match, tumorGTV_Match, lnGTV_Match

In [ ]:
#Hard coding labels, dont change
# ITV Labels
itv_tot_labels = ["ITV", "IGTV", "IgTV", "ITV1", "ITV2", "ITV3", "ITV totaal def", "2ITV", "ITV_TOT", "ITV_6000", "ITV_5100", "ITV_Totaal", "ITV_LBK", "ITV_LOK", "IGTV_6000"]
itv_tumor_labels = ["ITVtumor", "ITV_tumor", "ITVtumor def", "2ITV_tumor", "ITV-P", "ITVtumorA1", "ITV_tumor_LBK", "ITVtu", "IGTVp"]
itv_ln_labels = ["ITVklieren", "ITV_klier", "ITV_Klier", "ITVklieren def", "2ITV_klier", "ITV_n", "ITV_klier_LBK", "IGTVnode"]

# GTV Labels
gtv_tot_labels = ["GTV", "GTV1", "GTV2", "GTV3", "GTV totaal def", "2GTV", "GTV_TOT", "GTV_6000", "GTV_5100", "GTV_Totaal", "GTV_LBK", "GTV_LOK", "GTV_5000", "GTV_preop", "GTV_totaal", "GTV_voor OK", "GTVtotaal", "GTVnew", "GTV(op exp)", "GTVop", "GTV_voor OK", "GTV oorspr"]
gtv_tumor_labels = ["GTVtumor", "GTV_tumor", "GTVtumor def", "2GTV_tumor", "GTV-P", "GTVtumorA1", "GTV_tumor_LBK", "GTV tumor", "GTVp", "GTVtu", "GTVtumor_exp", "GTVtumor_exp_50->0", "GTVtumor_insp", "GTVpatelectasebewaren(1)", "GTVt"]
gtv_ln_labels = ["GTVklieren", "GTV_klier", "GTV_Klier", "GTVklieren def", "2GTV_klier", "GTV_n", "GTV_klier_LBK", "GTVn", "GTVnode_exp", "GTVnodes_exp", "GTVnodes_exp_50->0", "GTVnodes_insp", "GTVklieren"]

fieldnames=['ID', 'CT', 'ITVTot', 'ITVTumor', 'ITVLN', 'GTVtot', 'GTVTumor', 'GTVLN']


In [ ]:
#GET PATHS AND LABELS
#Where to look for file
root_path  = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/"

#Where the results will be saved (CSV) [@Robert->Change this so it saves in your folder]
csv_file_path = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/TestCSV_Delete.csv"

#Patient list  to look for in txt [@Robert->Change this so it points to the list of patients you care about]
PxListPath = '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/MissingCTRTPx_Feb29.txt'
PxList = pd.read_csv(PxListPath,header=None,dtype=str)


In [ ]:
#Look for patients already been analized (Ignore if this is the first time you run it)
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    checked_list = df['ID'].tolist()
    print(len(checked_list),"Px already checked,",len(PxList),"Missing")
else:
    checked_list = []

In [ ]:
#MAIN 
with open(csv_file_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if csvfile.tell() == 0:
        writer.writeheader()


    for Px in PxList[0]:
        strPx = str(Px)
        strPx = strPx.zfill(7)
        Px = strPx
        AvgCt_fpaths = []
        RTStruct_fpaths = []

        if int(Px) in checked_list:
            print(Px,"Px already checked")
        else:
            for root, dirs, files in os.walk(root_path+Px, topdown=True):
                for d in dirs:
                    if "Thorax" in d or "Ave" in d or "MIP" in d: #("CT" in d and not "RTSTRUCT" in d): #"%" in d or 
                        AvgCt_fpaths.append(root+'/'+d+'/')
                    if ('RTSTRUCT' in d or "pproved" in d) or ("PinnPlan" in d and not("RTDOSE" in d)):
                        RTStruct_fpaths.append(root+'/'+d+'/')


            print("Px",Px,"Num CTs",len(AvgCt_fpaths),"Num RTs",len(RTStruct_fpaths))

            ctMatch,rtMatch_ITVtot,rtMatch_ITVtumor,rtMatch_ITVln,rtMatch_GTVtot,rtMatch_GTVtumor,rtMatch_GTVln = GetTheContourRTMatch(AvgCt_fpaths,RTStruct_fpaths,itv_tot_labels,itv_tumor_labels,itv_ln_labels,gtv_tot_labels,gtv_tumor_labels,gtv_ln_labels)

            if len(ctMatch) == 0 :
                print("No Match")
            else: 
                print("Yes Match")
                
                writer.writerow({'ID':Px,'CT':ctMatch,'ITVTot':rtMatch_ITVtot,'ITVTumor':rtMatch_ITVtumor,'ITVLN':rtMatch_ITVln,'GTVtot':rtMatch_GTVtot,'GTVTumor':rtMatch_GTVtumor,'GTVLN':rtMatch_GTVln})
